In [1]:
# Import the necessary libraries
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
#Import data in numpy array format
data = np.genfromtxt('Data/data1.csv', delimiter=',')
data = data[1:,:]
data.shape

(100, 3)

In [3]:
#Check imported DATA
data[0:6,:]

array([[  4.17022 ,   7.203245, 153.916228],
       [  3.023326,   1.467559,  33.652484],
       [  1.862602,   3.455607,  68.218691],
       [  5.388167,   4.191945, 101.150179],
       [  2.044522,   8.781174, 174.900875],
       [  6.704675,   4.173048, 107.101716]])

In [4]:
X = data[:,0:2]
y = data[:,2:3]

In [5]:
scaler = StandardScaler()
X = scaler.fit_transform(X)
X = X.T
y = y.T
print ("X shape: " + str(X.shape))
print ("y shape: " + str(y.shape))

X shape: (2, 100)
y shape: (1, 100)


In [6]:
def relu(Z):
    r = np.maximum(0, Z)
    return r

def relu_backward(dA, Z):
    dZ = np.array(dA, copy=True) 
    dZ[Z <= 0] = 0    
    return dZ

In [7]:
def heInitialization(dim):
    w = np.random.randn(dim, 1)*np.sqrt(2/dim)
    b = 0
    return w, b

In [8]:
def propagate(w, b, X, Y, lambd):
    
    m = X.shape[1]
    
    # FORWARD PROPAGATION (FROM X TO COST)
    Z = np.dot(w.T, X)+b
    A = relu(Z)                                  
    mse_cost = 1/m*np.sum(np.power((Y-A), 2), axis=1)
    L2_regularization_cost = lambd/(2*m)*np.sum(np.square(w))
    cost = mse_cost + L2_regularization_cost

    
    # BACKWARD PROPAGATION (TO FIND GRAD)
    dA = -2*(Y - A)
    dZ = relu_backward(dA, Z)
    dw = 1/m*np.dot(X, dZ.T) + lambd/m*w
    db = 1/m*np.sum(dZ, axis=1)
    
    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    #assert(cost.shape == ())
    cost = np.squeeze(cost)
    
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost

In [9]:
def optimize(w, b, X, Y, num_iterations, learning_rate, lambd, print_cost = False):
    costs = []
    for i in range(num_iterations):
        
        # Cost and gradient calculation 
        grads, cost = propagate(w, b, X, Y, lambd)
        
        # Retrieve derivatives from grads
        dw = grads["dw"]
        db = grads["db"]
        
        # update rule (≈ 2 lines of code)
        w = w - learning_rate*dw
        b = b - learning_rate*db
        
        # Record the costs
        if i % 100 == 0:
            costs.append(cost)
        
        # Print the cost every 100 training iterations
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

In [10]:
def model(X_train, Y_train, num_iterations = 100000, learning_rate = 0.5, lambd=0.8, print_cost = False):
    
    w, b = heInitialization(X_train.shape[0])
    
    parameters, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, lambd, print_cost)
    
    w = parameters["w"]
    b = parameters["b"]
    
    d = {"costs": costs, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

In [11]:
d = model(X, y, num_iterations = 10000, learning_rate = 1, lambd= 0, print_cost = True)

Cost after iteration 0: 19578.998517
Cost after iteration 100: 13740.796800
Cost after iteration 200: 14071.316722
Cost after iteration 300: 10411.296468
Cost after iteration 400: 13670.081173
Cost after iteration 500: 13616.541414
Cost after iteration 600: 12663.553728
Cost after iteration 700: 11157.960420
Cost after iteration 800: 13789.654030
Cost after iteration 900: 13073.445831
Cost after iteration 1000: 9698.824838
Cost after iteration 1100: 13599.454538
Cost after iteration 1200: 12172.679734
Cost after iteration 1300: 9161.710733
Cost after iteration 1400: 12909.715299
Cost after iteration 1500: 13428.231410
Cost after iteration 1600: 11129.790481
Cost after iteration 1700: 13756.546664
Cost after iteration 1800: 11947.695838
Cost after iteration 1900: 9695.958303
Cost after iteration 2000: 13410.934868
Cost after iteration 2100: 12795.138266
Cost after iteration 2200: 12096.123679
Cost after iteration 2300: 12035.512993
Cost after iteration 2400: 12726.577818
Cost after iter

In [12]:
d["w"]

array([[ 28.49157392],
       [120.50573047]])

In [13]:
d["b"]

array([213.94984688])